In [8]:
import os

import gdown
import zipfile

from cvClassifier import logger
from cvClassifier.utils.common import get_size, read_yaml, create_directories 
from cvClassifier.constants import *

os.chdir("../")

In [3]:
%pwd

'/Users/riyosha/UW_MSDS/projects/End-to-end-chest-cancer-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [6]:
print(CONFIG_FILE_PATH)

config/config.yaml


In [17]:
class ConfigurationManager:
    # this class manages the configuration of the project

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        ''' Gets the config details for the data ingestion pipeline '''
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
    

In [21]:
class DataIngestion: 
    # this class is helps with all the data ingestion related tasks

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        ''' Downloads data from source_url'''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Successfully downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        ''' Extracts data from zip file '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f'Successfully extracted zip file to {unzip_path}')

In [14]:
%cd end-to-end-chest-cancer-classification

/Users/riyosha/UW_MSDS/projects/End-to-end-chest-cancer-classification


In [22]:
# Implementing the data ingestion pipeline!!
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-28 12:08:43,412: INFO: common]: yaml file successfully load from config/config.yaml
[2025-06-28 12:08:43,415: INFO: common]: yaml file successfully load from params.yaml
[2025-06-28 12:08:43,417: INFO: common]: directory created at: artifacts
[2025-06-28 12:08:43,418: INFO: common]: directory created at: artifacts/data_ingestion
[2025-06-28 12:08:43,418: INFO: 4278008370]: Downloading data from https://drive.google.com/file/d/1leEwKvCHlFMhQPlFUr8XQoSxp-79Qj_V/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1leEwKvCHlFMhQPlFUr8XQoSxp-79Qj_V
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1leEwKvCHlFMhQPlFUr8XQoSxp-79Qj_V&confirm=t&uuid=994724b6-9dfa-4ec8-86b4-87f342729cc9
To: /Users/riyosha/UW_MSDS/projects/End-to-end-chest-cancer-classification/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:04<00:00, 26.8MB/s] 

[2025-06-28 12:08:49,239: INFO: 4278008370]: Successfully downloaded data from https://drive.google.com/file/d/1leEwKvCHlFMhQPlFUr8XQoSxp-79Qj_V/view?usp=sharing into file artifacts/data_ingestion/data.zip


[2025-06-28 12:08:49,872: INFO: 4278008370]: Successfully extracted zip file to artifacts/data_ingestion
